In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator
#Importing the pre-trained model
from tensorflow.keras.applications.xception import Xception, preprocess_input, decode_predictions                     # the actual model
 

In [ ]:
#This final Model will be trained with larger size images so it can better recognize them. 

image_size = (299,299)
batch_size = 32

In [ ]:
train_gen = ImageDataGenerator(
    shear_range = 10.0,
    zoom_range = 0.2,
    horizontal_flip = True,
    preprocessig_function = preprocess_input
)

validation_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

In [ ]:
train_ds = train_gen.flow_from_directory(
    r"C:\Users\oliva\Documents\GitHub\ML-Projects\Neural Networks\clothing-dataset-small-master\clothing-dataset-small-master\train",
    target_size = image_size,
    batch_size = batch_size    
)

In [ ]:

val_ds = validation_gen.flow_from_directory(
    r"C:\Users\oliva\Documents\GitHub\ML-Projects\Neural Networks\clothing-dataset-small-master\clothing-dataset-small-master\validation",
    target_size = image_size,
    batch_size = batch_size
)

In [ ]:
def make_model( learning_rate):
    base_model = Xception(
        weights = 'imagenet',
        input_shape = (299,299,3),
        include_top = False
    )
    
    base_model.trainable = False
    
    inputs = keras.Input(shape=(299,299,3))
    base = base_model(inputs, training = False)
    vector = keras.layers.GlobalAveragePooling2D()(base)

    inner = keras.layers.Dense(100, activation='relu')(vector)                                       
    drop = keras.layers.Dropout(0.2)(inner)                                                                 
    outputs = keras.layers.Dense(10)(drop)                                                           
    model = keras.Model(inputs, outputs)

    optimizer = keras.optimizers.Adam(learning_rate)
    loss = keras.losses.CategoricalCrossentropy(from_logits=True)

    model.compile(optimizer=optimizer,
                loss= loss,
                metrics = ['accuracy'])
    return model